In [4]:
import pandas as pd
import os
import numpy as np
import nltk

# RA Code Cleaning

This notebook is meant to combine individual culture codings from the research assistants and do small exploratory analysis on them. This file is meant to be used within the original dropbox otherwise it will not work. If you are viewing this file from a GitHub repository, know that the files this notebook references are not meant to be accessed and that this notebook is merely included for the ease of analysis (and should be removed from the github later)

In [6]:
# Load and Append dataframe for all finished cultures by RA's
RA_list = ["AH","HL","KK","KY","LG","YM"]
localDir = '../../MEM-DEV-RA/Coding/'

df = pd.DataFrame()
count = 0
for RA in RA_list:
    # Extract unfinished 
    df_RA_unfin = pd.DataFrame()
    directory = localDir + RA
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and not f.__contains__("Eric") and f.endswith(".xlsx") :
            df_culture = pd.read_excel(f, header=[0,1])
            # Mark if 1st of 2nd run
            if f.__contains__("_2ndRun"):
                df_culture[("CODER","Run_Number")] = 2
            else:
                df_culture[("CODER","Run_Number")] = 1
            df_RA_unfin = pd.concat([df_RA_unfin, df_culture], ignore_index=True)
            count +=1
            print(f"Used: {f}")

    
    if len(df_RA_unfin) > 0:
        df_RA_unfin[("CODER","Finished")] = False

    # Extract Finished
    directory += '/Finished'
    df_RA_fin = pd.DataFrame()
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and not f.__contains__("Eric") and f.endswith(".xlsx") :
            df_culture = pd.read_excel(f, header=[0,1])
            # Mark if 1st of 2nd run
            if f.__contains__("_2ndRun"):
                df_culture[("CODER","Run_Number")] = 2
            else:
                df_culture[("CODER","Run_Number")] = 1
            df_RA_fin = pd.concat([df_RA_fin, df_culture], ignore_index=True)
            count +=1
            print(f"Used: {f}")
    if len(df_RA_fin) > 0:
        df_RA_fin[("CODER","Finished")] = True

    # concatinate the finished and unfinished then set the column by name
    df_RA_fin = pd.concat([df_RA_fin, df_RA_unfin], ignore_index=True)
    df_RA_fin[("CODER","Coder")] = RA
    df = pd.concat([df, df_RA_fin], ignore_index=True)
    
# Drop empty first column and empty first row(s)
df = df.drop(columns=df.columns[0])
df = df.dropna(subset=[("CULTURE","Passage Number")])
print(f"Unfinished Cultures: {len(set(df[~df[('CODER','Finished')]][('CULTURE','Culture')]))}")
print(f"Finished Cultures: {len(set(df[df[('CODER','Finished')]][('CULTURE','Culture')]))}")
print(f"DATAFRAME ROWS: {len(df)}")

Used: ../../MEM-DEV-RA/Coding/AH/Mataco.xlsx
Used: ../../MEM-DEV-RA/Coding/AH/Taiwan Hokkien.xlsx
Used: ../../MEM-DEV-RA/Coding/AH/Finished/Blackfoot.xlsx
Used: ../../MEM-DEV-RA/Coding/AH/Finished/Saami.xlsx
Used: ../../MEM-DEV-RA/Coding/HL/Finished/Lozi.xlsx
Used: ../../MEM-DEV-RA/Coding/HL/Finished/Yakut.xlsx
Used: ../../MEM-DEV-RA/Coding/HL/Finished/Aymara.xlsx
Used: ../../MEM-DEV-RA/Coding/HL/Finished/Somali.xlsx
Used: ../../MEM-DEV-RA/Coding/HL/Finished/Bororo.xlsx
Used: ../../MEM-DEV-RA/Coding/HL/Finished/Trobriands.xlsx
Used: ../../MEM-DEV-RA/Coding/KK/Finished/Aranda.xlsx
Used: ../../MEM-DEV-RA/Coding/KK/Finished/Tzeltal.xlsx
Used: ../../MEM-DEV-RA/Coding/KY/Iban.xlsx
Used: ../../MEM-DEV-RA/Coding/KY/Finished/Ganda.xlsx
Used: ../../MEM-DEV-RA/Coding/KY/Finished/Maasai.xlsx
Used: ../../MEM-DEV-RA/Coding/KY/Finished/Hausa.xlsx
Used: ../../MEM-DEV-RA/Coding/LG/Libyan Bedouin_2ndRun.xlsx
Used: ../../MEM-DEV-RA/Coding/LG/Lozi_2ndRun.xlsx
Used: ../../MEM-DEV-RA/Coding/LG/Finished/Kur

In [7]:
# Check No_info for counts
df[("EVENT","No_Info")].value_counts(dropna=False)

(EVENT, No_Info)
0      1659
NaN     385
1       285
/        64
Name: count, dtype: int64

In [8]:
#remove "/" and NaN rows
df = df[df[("EVENT","No_Info")] != "/"]
# remove all rows which have empty "No_Info" Note that some of these rows were recoded by EC if they ahd partial information enough to glean. Check comments
df = df.dropna(subset=[("EVENT","No_Info"), ("CAUSE","No_Info"), ("ACTION","No_Info")]) 
df[("EVENT","No_Info")].value_counts(dropna=False)

(EVENT, No_Info)
0    1654
1     284
Name: count, dtype: int64

In [9]:
multiCol = list(df.columns)
valuecountCol = []
for col in multiCol:
    if col[0] in ["CULTURE", "OTHER", "CODER"] or col[1] in ["Description", "Local_terms", "Local_Terms"]:
        continue
    else:
        valuecountCol.append(col)

# look through and make sure each column only has a 1 or 0
for col in valuecountCol:
    Counts = df[col].value_counts()
    if len(Counts) >2:
        print(Counts)
    else:
        print(f"{col}  {(45 - len(str(col))) * ' '}CORRECTLY contains ONLY 2 values")


('EVENT', 'No_Info')                           CORRECTLY contains ONLY 2 values
('EVENT', 'Illness')                           CORRECTLY contains ONLY 2 values
('EVENT', 'Accident')                          CORRECTLY contains ONLY 2 values
('EVENT', 'Other')                             CORRECTLY contains ONLY 2 values
('CAUSE', 'No_Info')                           CORRECTLY contains ONLY 2 values
('CAUSE', 'Just_Happens')                      CORRECTLY contains ONLY 2 values
('CAUSE', 'Material_Physical')                 CORRECTLY contains ONLY 2 values
('CAUSE', 'Spirits_Gods')                      CORRECTLY contains ONLY 2 values
('CAUSE', 'Witchcraft_Sorcery')                CORRECTLY contains ONLY 2 values
('CAUSE', 'Rule_Violation_Taboo')              CORRECTLY contains ONLY 2 values
('CAUSE', 'Jealousy_Evil_Eye')                 CORRECTLY contains ONLY 2 values
('ACTION', 'No_Info')                          CORRECTLY contains ONLY 2 values
('ACTION', 'Physical_Material')         

In [10]:
# Save mistakes in a dataframe and check
df_mistakes = pd.DataFrame()
for col in valuecountCol:
    df_mistakesRA = df[(df[col] != 1) & (df[col] != 0)]
    df_mistakes = pd.concat([df_mistakes, df_mistakesRA], ignore_index=True)
df_mistakes.sort_values(by=[("CODER","Coder"), ("CULTURE","Passage Number")], ascending=True)[ [("CULTURE","Culture"), ("CULTURE","Passage Number")]+ valuecountCol + [("CODER","Coder")]]

Empty DataFrame
Columns: [(CULTURE, Culture), (CULTURE, Passage Number), (EVENT, No_Info), (EVENT, Illness), (EVENT, Accident), (EVENT, Other), (CAUSE, No_Info), (CAUSE, Just_Happens), (CAUSE, Material_Physical), (CAUSE, Spirits_Gods), (CAUSE, Witchcraft_Sorcery), (CAUSE, Rule_Violation_Taboo), (CAUSE, Jealousy_Evil_Eye), (ACTION, No_Info), (ACTION, Physical_Material), (ACTION, Technical_Specialist), (ACTION, Divination), (ACTION, Shaman_Medium_Healer), (ACTION, Priest_High_Religion), (ACTION, Other), (CODER, Coder)]
Index: []

[0 rows x 21 columns]

20 values were corrected for typing errors and interpretated based on Research assistant descriptions and Research manager's final word. a good number were merely NaN values and were recoded as being 0's <br>
Rows were also removed which have a missing no_info value as these tended to be completely blank <br>
Codings found that were clearly wrong were not chnaged for the sake of not interfering with coder's reliability or decision.

In [11]:
df.head(5)

CULTURE                                                 
  Passage Number         Region           SubRegion    Culture   
1         5153.0  North-America  Plains and Plateau  Blackfoot  \
4         5206.0  North-America  Plains and Plateau  Blackfoot   
5         5208.0  North-America  Plains and Plateau  Blackfoot   
6         5216.0  North-America  Plains and Plateau  Blackfoot   
7         5217.0  North-America  Plains and Plateau  Blackfoot   

                                                                      
                                      DocTitle              Section   
1  Ceremonial bundles of the Blackfoot Indians                ( e )  \
4  Ceremonial bundles of the Blackfoot Indians     Untitled Section   
5  Ceremonial bundles of the Blackfoot Indians     Untitled Section   
6  Ceremonial bundles of the Blackfoot Indians  Origins of Rituals.   
7  Ceremonial bundles of the Blackfoot Indians         War Bridles.   

                                            
                      Author Page    Year   
1  Wissler, Clark, 1870-1947   74  1912.0  \
4  Wissler, Clark, 1870-1947   88  1912.0   
5  Wissler, Clark, 1870-1947   90  1912.0   
6  Wissler, Clark, 1870-1947  101  1912.0   
7  Wissler, Clark, 1870-1947  110  1912.0   

                                                      ...     ACTION   
                                                 OCM  ... Divination   
1         ['159', '493', '751', '756', '776', '778']  ...          0  \
4         ['177', '436', '750', '756', '778', '832']  ...          0   
5                ['177', '609', '753', '755', '761']  ...          0   
6         ['121', '150', '159', '174', '753', '787']  ...          0   
7  ['224', '276', '283', '302', '721', '752', '75...  ...          0   

                                                    
  Shaman_Medium_Healer Priest_High_Religion Other   
1                    0                    0     0  \
4                    0                    0     0   
5                    0                    0     1   
6                    0                    0     0   
7                    1                    0     0   

                                                                   
                                         Description Local_terms   
1  narrator recalls seing in a dream that by usin...           0  \
4   injury by Cree medicine can be removed by oth...           0   
5  preemptive action: treat all visiting Cree car...           0   
6         no preemptive or consequent actions stated           0   
7  horse medicinemen use their powers to prevent ...           0   

                                               OTHER      CODER                 
                                      Other_Comments Run_Number Finished Coder  
1  not sure how to code this one, misfortune is n...          1     True    AH  
4                                                  0          1     True    AH  
5  not sure if "treating someone carefully" is co...          1     True    AH  
6                                                  0          1     True    AH  
7                                                  0          1     True    AH  

[5 rows x 40 columns]

In [12]:
# Calculate response bias (unbalanced 1 vs. 0, a perfectly balanced one would be .5 or 50%)
def colProportion(df, col, present=1):
    value_counts = df[col].value_counts()
    proportion = round(value_counts[present]/len(df),2)
    return proportion

print("BIAS FOR ANSWERING \'PRESENT\'")
for col in valuecountCol:
    if col[1] == "No_Info":
        proportion = colProportion(df, col, present=0)
        print(f"\n{col[0]}:{(38-len(col[0]))*' '}{proportion}")
    else:
        proportion = colProportion(df, col, present=1)
        print(f"\t{col[1]}:{(30-len(col[1]))*' '}{proportion}")



BIAS FOR ANSWERING 'PRESENT'

EVENT:                                 0.85
	Illness:                       0.66
	Accident:                      0.07
	Other:                         0.27

CAUSE:                                 0.68
	Just_Happens:                  0.06
	Material_Physical:             0.24
	Spirits_Gods:                  0.35
	Witchcraft_Sorcery:            0.12
	Rule_Violation_Taboo:          0.12
	Jealousy_Evil_Eye:             0.03

ACTION:                                0.68
	Physical_Material:             0.41
	Technical_Specialist:          0.08
	Divination:                    0.06
	Shaman_Medium_Healer:          0.18
	Priest_High_Religion:          0.03
	Other:                         0.17


In [13]:
from datetime import date
today = date.today() #get date


# Passages per hour list (Includes finished and unfinished)
PPH_list = []
df_RA = pd.DataFrame({"RA":[], "Passages":[], "Hours":[], 'PassPerHour':[], 'Words':[], 'WordsPerHour':[], 'Date':[]})
hours_dict = {'LG':48.5, 'AH':52.1, 'KK':27.9, 'HL':57.2, 'YM':97.6, 'KY':40.4}
code_count = df[("CODER","Coder")].value_counts()
for key, val in code_count.items():
    PPH = round(val/hours_dict[key], 2)
    PPH_list.append(PPH)
    df_word_Count = df.loc[df[("CODER","Coder")]==key]
    word_count = df_word_Count[("CULTURE","Passage")].str.split().str.len().sum()
    WPH = round(word_count/hours_dict[key], 2)
    print(f"{key} {val}{(4-len(str(val)))*' '}{PPH} Passages per hour         {word_count}{(8-len(str(word_count)))*' '}{WPH}{(8-len(str(WPH)))*' '} Words Per Hour")
    df_RA.loc[len(df_RA.index)] = [key, val, hours_dict[key], PPH, word_count, WPH, today ]
print(f"mean speed {round(np.mean(PPH_list),2)} Passages per hour")

YM 530 5.43 Passages per hour         74398   762.27   Words Per Hour
LG 371 7.65 Passages per hour         65884   1358.43  Words Per Hour
HL 312 5.45 Passages per hour         48172   842.17   Words Per Hour
KY 294 7.28 Passages per hour         59371   1469.58  Words Per Hour
AH 287 5.51 Passages per hour         48859   937.79   Words Per Hour
KK 144 5.16 Passages per hour         20049   718.6    Words Per Hour
mean speed 6.08 Passages per hour


In [14]:
# dummy code for later use
# df_word_Count[("CULTURE","Passage")].str.split().str.len()

In [15]:
# Save Progress

fileName = "RA_Progress.xlsx"


if os.path.isfile(fileName):
    df_RA_old = pd.read_excel(fileName, index_col=False)
    df_RA = pd.concat([df_RA, df_RA_old], ignore_index=True)
    df_RA = df_RA.astype({'Date':'datetime64[ns]'})
    df_RA = df_RA.sort_values(by=['Date','RA'], ascending=[True,False])
    df_RA = df_RA.drop_duplicates(subset=["RA","Date"], keep="last") #may work to allow updating the old rows, not sure, haven't tested
    df_RA = df_RA.reset_index(drop=True)
df_RA.to_excel(fileName, index=False)


In [16]:
# Counts
print(f"Unfinished but started (unique) Cultures: {len(set(df[~df[('CODER','Finished')]][('CULTURE','Culture')]))}")
print(f"Unfinished Culture Passage count: {len(df[~df[('CODER','Finished')]][('CULTURE','Culture')])}\n")
print(f"Finished (unique) Cultures: {len(set(df[df[('CODER','Finished')]][('CULTURE','Culture')]))}")
print(f"Finished Culture Passage Count: {len(df[df[('CODER','Finished')]][('CULTURE','Culture')])}\n")
print(f"Total (unique) Cultures: {len(set(df[('CULTURE','Culture')]))}")
print(f"Total Passages: {len(df)}")

Unfinished but started (unique) Cultures: 5
Unfinished Culture Passage count: 252

Finished (unique) Cultures: 26
Finished Culture Passage Count: 1686

Total (unique) Cultures: 29
Total Passages: 1938


In [17]:
print(f"1st Run Cultures: {len(set(df[df[('CODER','Run_Number')]==1][('CULTURE','Culture')]))}")
print(f"1st Run Culture Passage Count: {len(df[df[('CODER','Run_Number')]==1][('CULTURE','Culture')])}\n")
print(f"2nd Run Cultures: {len(set(df[df[('CODER','Run_Number')]==2][('CULTURE','Culture')]))}")
print(f"2nd Run Culture Passage Count: {len(df[df[('CODER','Run_Number')]==2][('CULTURE','Culture')])}\n")

1st Run Cultures: 29
1st Run Culture Passage Count: 1890

2nd Run Cultures: 3
2nd Run Culture Passage Count: 48



In [18]:
# Save concatinated file
df.to_excel("Culture_Coding.xlsx")

## Inter-Rater Reliability Coding

Check agreement between coders

In [19]:
# Cultures
df_one = df.loc[df[("CODER","Run_Number")]==1]
print(df_one[("CULTURE","Culture")].unique())
df_two = df.loc[df[("CODER","Run_Number")]==2]
print(df_two[("CULTURE","Culture")].unique())


['Blackfoot' 'Saami' 'Mataco' 'Taiwan Hokkien' 'Lozi' 'Yakut' 'Aymara'
 'Somali' 'Bororo' 'Trobriands' 'Aranda' 'Tzeltal' 'Ganda' 'Maasai'
 'Hausa' 'Iban' 'Kurds' 'Tukano' 'Yanoama' 'Copper Inuit' 'Santal' 'Kogi'
 'Pawnee' 'Ona' 'Chukchee' 'Libyan Bedouin' 'Andamans' 'Lau Fijians'
 'Tikopia']
['Lozi' 'Copper Inuit' 'Pawnee']


In [73]:
df_dummy = df[df.duplicated(subset=("CULTURE","Passage Number"), keep=False)]

#Convert the passage numbers to the index
df_dummy.loc[:, "Passage Number"] = df_dummy[("CULTURE", "Passage Number")]
df_dummy = df_dummy.drop(columns=[("CULTURE", "Passage Number")])
df_dummy = df_dummy.set_index("Passage Number")

# remove extraneous columns
df_dummy = df_dummy.drop(columns=df_dummy.columns[df_dummy.columns.get_level_values(0).isin(['CULTURE','OTHER'])])
df_dummy = df_dummy.iloc[:, :-2] # for some reason this does not allow me to drop all of the CODER column
rem_cols = [i for i in df_dummy.columns if i[1] in ['Description', 'Local_Terms', 'Local_terms', 'Other_Comments']]
df_dummy = df_dummy.drop(columns=rem_cols)

# check if number match above numbers
df_dummy[("CODER","Run_Number")].value_counts()

/var/folders/r8/j6vx966d5rj6srtb4x895kvr0000gp/T/ipykernel_74508/2340595576.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dummy.loc[:, "Passage Number"] = df_dummy[("CULTURE", "Passage Number")]


(CODER, Run_Number)
1    47
2    47
Name: count, dtype: int64

In [78]:
df_one = df_dummy.loc[df_dummy[("CODER","Run_Number")]==1]
df_two = df_dummy.loc[df_dummy[("CODER","Run_Number")]==2]

df_comparison = df_one == df_two
df_comparison.mean(axis=0)



EVENT   No_Info                 0.893617
        Illness                 0.829787
        Accident                0.808511
        Other                   0.468085
CAUSE   No_Info                 0.765957
        Just_Happens            0.744681
        Material_Physical       0.702128
        Spirits_Gods            0.787234
        Witchcraft_Sorcery      0.851064
        Rule_Violation_Taboo    0.851064
        Jealousy_Evil_Eye       0.957447
ACTION  No_Info                 0.808511
        Physical_Material       0.574468
        Technical_Specialist    0.851064
        Divination              0.893617
        Shaman_Medium_Healer    0.914894
        Priest_High_Religion    1.000000
        Other                   0.617021
CODER   Run_Number              0.000000
dtype: float64